In [1]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
reviews_df = pd.read_excel('data/reviews.xlsx')
reviews_df.head()

In [4]:
vader = SentimentIntensityAnalyzer()

In [5]:
def vader_scores(text: str):
    s = vader.polarity_scores(str(text))
    compound = s["compound"]
    if compound >= 0.05:
        label = "Positive"
    elif compound <= -0.05:
        label = "Negative"
    else:
        label = "Neutral"
    return pd.Series([s["neg"], s["neu"], s["pos"], compound, label])

In [6]:
results = reviews_df[["VADER_neg", "VADER_neu", "VADER_pos", "VADER_compound", "VADER_label"]] = (
    reviews_df["Review Text"].apply(vader_scores)
)
print(results)

        0      1      2       3         4
0   0.000  0.690  0.310  0.9521  Positive
1   0.096  0.731  0.173  0.2144  Positive
2   0.119  0.881  0.000 -0.5665  Negative
3   0.000  0.596  0.404  0.9622  Positive
4   0.057  0.885  0.058  0.0049   Neutral
5   0.000  0.772  0.228  0.8570  Positive
6   0.070  0.877  0.053 -0.1533  Negative
7   0.000  0.668  0.332  0.9318  Positive
8   0.000  0.819  0.181  0.6369  Positive
9   0.000  0.655  0.345  0.9348  Positive
10  0.000  0.651  0.349  0.8818  Positive
11  0.168  0.751  0.081 -0.4391  Negative
12  0.000  0.593  0.407  0.9287  Positive
13  0.000  0.876  0.124  0.3716  Positive
14  0.000  0.687  0.313  0.8020  Positive
15  0.000  0.862  0.138  0.4939  Positive
16  0.000  0.581  0.419  0.7574  Positive
17  0.000  0.728  0.272  0.8622  Positive
18  0.129  0.813  0.058 -0.3163  Negative
19  0.000  0.577  0.423  0.9446  Positive
20  0.000  1.000  0.000  0.0000   Neutral
21  0.000  0.580  0.420  0.9062  Positive
22  0.168  0.832  0.000 -0.3449  N